<a href="https://colab.research.google.com/github/Akr-ds/chatbot-using-rag/blob/main/rag_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# clone the github repository to fetch the pdf file "business_for_dummies"
!git clone https://github.com/Akr-ds/chatbot-using-rag.git

Cloning into 'chatbot-using-rag'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 5.95 MiB | 14.53 MiB/s, done.


In [3]:
#install dependencies needed for making qa bot
!pip install \
   langchain_community \
   langchain_pinecone \
   langchain_openai \
   unstructured \
   langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 57.5 MB/s eta 0:00:00
  Preparing metadat

In [21]:
#import necessary libraries
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import glob
import requests
import fitz
import re


In [50]:
# defining the function for downloading the github url
def download_pdf(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(response.content)
# defining a function for extracting the text for text retrieval and generation
def get_text_from_pdf(file_path):
    text = ""
    doc = fitz.open(file_path)
    for page in doc:
        text += page.get_text()
    return text

# URL of the PDF file
pdf_url = "https://github.com/Akr-ds/chatbot-using-rag/blob/main/business%20_for_dummies.pdf?raw=true"
pdf_path = "business_for_dummies.pdf"

# extracting the pdf and its contents
download_pdf(pdf_url,pdf_path)
text_business=get_text_from_pdf(pdf_path)
text_business[:100]


'Part I: Getting Started\nBefore you get a business off the ground you have to do all the\npreliminary '

In [28]:
#cleaning the text for newline characters
clean_text=re.sub(r'\n',' ',text_business)
clean_text[0:100]

'Part I: Getting Started Before you get a business off the ground you have to do all the preliminary '

In [38]:
#importing openai library for necessary text embeddings
import openai

In [45]:
#setting up the openai api key and pinecone vector database , pinecone is use as a database for similarity search
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')


In [ ]:
# creating embeddings of the cleaned text and using recursive splitter from the pinecone db to chunk the text
embeddings=OpenAIEmbeddings(
    model='text-embedding-3-small'
)
index_name='busy-bot'

text_splitter=RecursiveCharacterTextSplitter()
docs=text_splitter.create_documents([clean_text])


In [48]:
#turn the chunked  text into a vector form to perform performing similarity search using pinecone db
vectorscore=PineconeVectorStore(docs,embeddings,index_name=index_name)

In [ ]:
#setting up query and searching for similar text needed for performing text generation
query=" tell me about the business"
similar_docs=vectorscore.similarity_search(query)